In [3]:
!pip install kaggle



   ------------- -------------------------- 1/3 [python-slugify]
   -------------------------- ------------- 2/3 [kaggle]
   -------------------------- ------------- 2/3 [kaggle]
   -------------------------- ------------- 2/3 [kaggle]
   -------------------------- ------------- 2/3 [kaggle]
   -------------------------- ------------- 2/3 [kaggle]
   -------------------------- ------------- 2/3 [kaggle]
   -------------------------- ------------- 2/3 [kaggle]
   -------------------------- ------------- 2/3 [kaggle]
   -------------------------- ------------- 2/3 [kaggle]
   -------------------------- ------------- 2/3 [kaggle]
   ---------------------------------------- 3/3 [kaggle]



# Importing Liberaries 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error
import xgboost as xgb
import shutil, os, zipfile


# Downloading & Extracting Data 

In [13]:

os.remove(r"C:\Users\Athar\.kaggle\kaggle.json")

os.makedirs(r"C:\Users\Athar\.kaggle", exist_ok=True)
shutil.copy(
    r"D:\ml_internship\3_sales_forecasting\data\.kaggle\kaggle.json",
    r"C:\Users\Athar\.kaggle\kaggle.json"
)
print(" Copied kaggle.json to the default location.")


 Copied kaggle.json to the default location.


In [10]:
!kaggle competitions download -c walmart-recruiting-store-sales-forecasting



  0%|          | 0.00/2.70M [00:00<?, ?B/s]
100%|##########| 2.70M/2.70M [00:00<00:00, 20.4MB/s]
100%|##########| 2.70M/2.70M [00:00<00:00, 17.3MB/s]


In [14]:



# Define paths
src = "walmart-recruiting-store-sales-forecasting.zip"  
dest_folder = r"D:\ml_internship\3_sales_forecasting\data"
dest_zip = os.path.join(dest_folder, "walmart-recruiting-store-sales-forecasting.zip")

# Make sure the destination folder exists
os.makedirs(dest_folder, exist_ok=True)

# Move the zip file to the data folder
shutil.move(src, dest_zip)
print(f"Moved to: {dest_zip}")

# Extract the ZIP file inside the data folder
with zipfile.ZipFile(dest_zip, 'r') as zip_f:
    zip_f.extractall(dest_folder)
print(f"Extracted contents to: {dest_folder}")


Moved to: D:\ml_internship\3_sales_forecasting\data\walmart-recruiting-store-sales-forecasting.zip
Extracted contents to: D:\ml_internship\3_sales_forecasting\data


In [15]:
data_folder = r"D:\ml_internship\3_sales_forecasting\data"

# Loop through all files in the data folder
for file in os.listdir(data_folder):
    if file.endswith(".zip"):
        zip_path = os.path.join(data_folder, file)
        extract_folder = data_folder  
        print(f"Extracting {file}...\n")
        
        with zipfile.ZipFile(zip_path, 'r') as zip_f:
            zip_f.extractall(extract_folder)
        print(f"Extracted {file}")

Extracting features.csv.zip...

Extracted features.csv.zip
Extracting sampleSubmission.csv.zip...

Extracted sampleSubmission.csv.zip
Extracting test.csv.zip...

Extracted test.csv.zip
Extracting train.csv.zip...

Extracted train.csv.zip
Extracting walmart-recruiting-store-sales-forecasting.zip...

Extracted walmart-recruiting-store-sales-forecasting.zip


In [16]:
for file in os.listdir(data_folder):
    if file.endswith(".zip"):
        os.remove(os.path.join(data_folder, file))
        print(f"Deleted {file}")

# Reading & Exploring Data 

In [20]:
train= pd.read_csv("../data/train.csv")
stores= pd.read_csv("../data/stores.csv")
features= pd.read_csv("../data/features.csv")

In [21]:
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [22]:
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [23]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  object 
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), object(1)
memory usage: 13.3+ MB


In [27]:
train.Store.value_counts(dropna= False)

Store
13    10474
10    10315
4     10272
1     10244
2     10238
24    10228
27    10225
34    10224
20    10214
6     10211
32    10202
19    10148
31    10142
28    10113
41    10088
11    10062
23    10050
14    10040
40    10017
15     9901
8      9895
39     9878
17     9864
18     9859
26     9854
25     9804
7      9762
12     9705
22     9688
45     9637
21     9582
35     9528
29     9455
16     9443
3      9036
5      8999
9      8867
38     7362
37     7206
44     7169
30     7156
42     6953
43     6751
33     6487
36     6222
Name: count, dtype: int64

In [28]:
train.Dept.value_counts(dropna=False)

Dept
1     6435
16    6435
92    6435
38    6435
40    6435
      ... 
78     235
77     150
65     143
39      16
43      12
Name: count, Length: 81, dtype: int64

In [29]:
train.IsHoliday.value_counts(dropna=False)

IsHoliday
False    391909
True      29661
Name: count, dtype: int64

In [30]:
stores.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Store   45 non-null     int64 
 1   Type    45 non-null     object
 2   Size    45 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 1.2+ KB


In [32]:
stores.Type.value_counts(dropna=False)

Type
A    22
B    17
C     6
Name: count, dtype: int64

In [34]:
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   object 
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), object(1)
memory usage: 712.0+ KB


In [44]:
print(train.shape)
print(stores.shape)

(421570, 5)
(45, 3)


In [ ]:
df= train.merge(stores, on='Store', how='left').merge(features, on=['Store', 'Date'], how='left')
print(df.shape)
print("\n")
df.head()